In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

print("Libraries loaded!")

Libraries loaded!


In [2]:
print("Current directory:", os.getcwd())
print("Excel files available:", [f for f in os.listdir('.') if f.endswith('.xlsx')])

Current directory: C:\Users\HP USER
Excel files available: []


In [3]:
# Let's first see what we're working with
print("CURRENT DATA STATUS:")
print(f"DataFrame name: df")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

CURRENT DATA STATUS:
DataFrame name: df


NameError: name 'df' is not defined

In [ ]:
file_path = r"C:\Users\HP USER\OneDrive\Desktop\SELECTED FOOD NOVEMBER 2022.xlsx"

try:
    df = pd.read_excel(file_path, sheet_name='SELECTED FOOD NOV 2022')
    print("File loaded successfully!")
    print(f"Shape: {df.shape}")
    display(df.head(3))
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Show the first few rows
print("\nFIRST 3 ROWS:")
display(df.head(3))

In [ ]:
# Let's examine the column structure carefully
print("COLUMN ANALYSIS:")
for i, col_name in enumerate(df.columns):
    print(f"Column {i}: '{col_name}'")

# Check the first column specifically
first_col = df.columns[0]
print(f"\nFirst column name: '{first_col}'")
print(f"First column values sample:")
print(df[first_col].head(10))

In [ ]:
def simple_clean_data(dataframe):
    """
    Simple and safe data cleaning
    """
    # Work with a copy
    cleaned = dataframe.copy()
    
    print(f"Original shape: {cleaned.shape}")
    
    # Remove completely empty rows
    cleaned = cleaned.dropna(how='all')
    print(f"After removing empty rows: {cleaned.shape}")
    
    # Remove completely empty columns
    cleaned = cleaned.dropna(axis=1, how='all')
    print(f"After removing empty columns: {cleaned.shape}")
    
    # Reset index
    cleaned = cleaned.reset_index(drop=True)
    
    return cleaned

# Apply the cleaning
df_clean = simple_clean_data(df)
print(f"Cleaned data shape: {df_clean.shape}")

In [ ]:
# Let's identify which columns contain prices
print("IDENTIFYING PRICE COLUMNS:")

# Show all columns with their first values
for i, col in enumerate(df_clean.columns):
    sample_value = df_clean[col].iloc[0] if len(df_clean) > 0 else "No data"
    print(f"{i:2d}. '{col}': {sample_value}")

# Based on your earlier message, let's assume these are the key columns:
# Column 0: Items
# Column 1: Nov-21 prices  
# Column 2: Oct-22 prices
# Column 3: Nov-22 prices

In [ ]:
def simple_inflation_analysis(clean_df):
    """
    Simple inflation analysis without complex assumptions
    """
    try:
        # Let's work with the first few columns that likely contain prices
        results = []
        
        for i in range(min(10, len(clean_df))):  # Analyze first 10 rows
            item_name = clean_df.iloc[i, 0]  # First column = items
            
            # Try to extract prices from columns 1, 2, 3
            try:
                price_2021 = pd.to_numeric(clean_df.iloc[i, 1], errors='coerce')
                price_2022 = pd.to_numeric(clean_df.iloc[i, 3], errors='coerce')
                
                if not pd.isna(price_2021) and not pd.isna(price_2022) and price_2021 > 0:
                    yoy_increase = price_2022 - price_2021
                    yoy_percent = (yoy_increase / price_2021) * 100
                    
                    results.append({
                        'Item': item_name,
                        'Nov_2021': price_2021,
                        'Nov_2022': price_2022,
                        'YoY_Increase': yoy_increase,
                        'YoY_Percent': yoy_percent
                    })
                    
            except Exception as e:
                continue
        
        return pd.DataFrame(results)
    
    except Exception as e:
        print(f"Error in analysis: {e}")
        return pd.DataFrame()

# Run the simple analysis
inflation_results = simple_inflation_analysis(df_clean)

if not inflation_results.empty:
    print("Inflation analysis successful!")
    print(f"Analyzed {len(inflation_results)} items")
    display(inflation_results.head(10))
else:
    print("No results from inflation analysis")
    print("Let's try a different approach...")

In [ ]:
if not inflation_results.empty:
    # Sort by highest inflation
    top_inflated = inflation_results.nlargest(10, 'YoY_Percent')
    
    # Create a simple bar chart
    plt.figure(figsize=(12, 8))
    bars = plt.barh(top_inflated['Item'], top_inflated['YoY_Percent'])
    plt.xlabel('Year-over-Year Inflation (%)')
    plt.title('Top 10 Most Inflated Food Items')
    plt.gca().invert_yaxis()
    
    # Add percentage labels
    for bar in bars:
        width = bar.get_width()
        plt.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()
    
    print("Top 5 Most Inflated Items:")
    display(top_inflated.head())

In [ ]:
# Let's analyze the top inflated items in more detail
print("DETAILED ANALYSIS OF TOP INFLATED ITEMS")
print("=" * 60)

if not inflation_results.empty:
    # Get top 10 most inflated items
    top_10 = inflation_results.nlargest(10, 'YoY_Percent')
    
    # Display with better formatting
    pd.set_option('display.float_format', '{:,.2f}'.format)
    
    print("Top 10 Most Inflated Items (Detailed):")
    detailed_top = top_10[['Item', 'Nov_2021', 'Nov_2022', 'YoY_Increase', 'YoY_Percent']].copy()
    detailed_top['Nov_2021'] = detailed_top['Nov_2021'].round(2)
    detailed_top['Nov_2022'] = detailed_top['Nov_2022'].round(2)
    detailed_top['YoY_Increase'] = detailed_top['YoY_Increase'].round(2)
    detailed_top['YoY_Percent'] = detailed_top['YoY_Percent'].round(2)
    
    display(detailed_top)
    
    # Summary statistics
    print(f"\nSUMMARY STATISTICS:")
    print(f"Average YoY Inflation: {inflation_results['YoY_Percent'].mean():.1f}%")
    print(f"Maximum YoY Inflation: {inflation_results['YoY_Percent'].max():.1f}%")
    print(f"Minimum YoY Inflation: {inflation_results['YoY_Percent'].min():.1f}%")
    print(f"Number of items analyzed: {len(inflation_results)}")
    
    # Count items by inflation category
    high_inflation = len(inflation_results[inflation_results['YoY_Percent'] > 20])
    medium_inflation = len(inflation_results[(inflation_results['YoY_Percent'] > 10) & (inflation_results['YoY_Percent'] <= 20)])
    low_inflation = len(inflation_results[inflation_results['YoY_Percent'] <= 10])
    
    print(f"\nINFLATION DISTRIBUTION:")
    print(f"High inflation (>20%): {high_inflation} items")
    print(f"Medium inflation (10-20%): {medium_inflation} items") 
    print(f"Low inflation (≤10%): {low_inflation} items")

In [ ]:
# Create more comprehensive visualizations
if not inflation_results.empty:
    # Create a figure with multiple subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
    
    # Plot 1: Top 10 Most Inflated Items
    top_10 = inflation_results.nlargest(10, 'YoY_Percent')
    bars1 = ax1.barh(top_10['Item'], top_10['YoY_Percent'], color='red', alpha=0.7)
    ax1.set_xlabel('YoY Inflation (%)')
    ax1.set_title('Top 10 Most Inflated Items', fontweight='bold', fontsize=14)
    ax1.invert_yaxis()
    for bar in bars1:
        width = bar.get_width()
        ax1.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontweight='bold')
    
    # Plot 2: Bottom 10 Least Inflated Items
    bottom_10 = inflation_results.nsmallest(10, 'YoY_Percent')
    bars2 = ax2.barh(bottom_10['Item'], bottom_10['YoY_Percent'], color='green', alpha=0.7)
    ax2.set_xlabel('YoY Inflation (%)')
    ax2.set_title('Top 10 Least Inflated Items', fontweight='bold', fontsize=14)
    ax2.invert_yaxis()
    for bar in bars2:
        width = bar.get_width()
        ax2.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontweight='bold')
    
    # Plot 3: Price Comparison for Top 5 Items
    top_5 = inflation_results.nlargest(5, 'YoY_Percent')
    x_pos = np.arange(len(top_5))
    width = 0.35
    
    ax3.bar(x_pos - width/2, top_5['Nov_2021'], width, label='Nov 2021', alpha=0.7)
    ax3.bar(x_pos + width/2, top_5['Nov_2022'], width, label='Nov 2022', alpha=0.7)
    ax3.set_xlabel('Items')
    ax3.set_ylabel('Price (₦)')
    ax3.set_title('Price Comparison: Nov 2021 vs Nov 2022', fontweight='bold', fontsize=14)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels([item[:15] + '...' if len(item) > 15 else item for item in top_5['Item']], rotation=45)
    ax3.legend()
    
    # Plot 4: Inflation Distribution Histogram
    ax4.hist(inflation_results['YoY_Percent'], bins=15, color='purple', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('YoY Inflation (%)')
    ax4.set_ylabel('Number of Items')
    ax4.set_title('Distribution of Inflation Rates', fontweight='bold', fontsize=14)
    ax4.axvline(inflation_results['YoY_Percent'].mean(), color='red', linestyle='--', 
                label=f'Mean: {inflation_results["YoY_Percent"].mean():.1f}%')
    ax4.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Let's categorize items and analyze by food type
def categorize_items(item_name):
    """Categorize food items into broad categories"""
    item_lower = str(item_name).lower()
    
    if any(word in item_lower for word in ['beef', 'chicken', 'fish', 'mackerel', 'sardine', 'tilapia']):
        return 'Animal Protein'
    elif any(word in item_lower for word in ['rice', 'beans', 'maize', 'wheat', 'flour']):
        return 'Grains & Staples'
    elif any(word in item_lower for word in ['oil', 'palm', 'groundnut', 'vegetable']):
        return 'Oils & Fats'
    elif any(word in item_lower for word in ['tomato', 'onion', 'plantain', 'yam', 'potato']):
        return 'Vegetables & Fruits'
    elif any(word in item_lower for word in ['egg', 'milk']):
        return 'Dairy & Eggs'
    elif any(word in item_lower for word in ['bread', 'gari']):
        return 'Processed Foods'
    else:
        return 'Other'

# Add categories to our inflation data
if not inflation_results.empty:
    inflation_results['Category'] = inflation_results['Item'].apply(categorize_items)
    
    # Analyze inflation by category
    category_analysis = inflation_results.groupby('Category').agg({
        'YoY_Percent': ['mean', 'max', 'min', 'count'],
        'Nov_2021': 'mean',
        'Nov_2022': 'mean'
    }).round(2)
    
    category_analysis.columns = ['Avg_YoY_%', 'Max_YoY_%', 'Min_YoY_%', 'Item_Count', 'Avg_Price_2021', 'Avg_Price_2022']
    category_analysis = category_analysis.sort_values('Avg_YoY_%', ascending=False)
    
    print("INFLATION BY FOOD CATEGORY:")
    display(category_analysis)
    
    # Visualization by category
    plt.figure(figsize=(12, 6))
    category_avg = inflation_results.groupby('Category')['YoY_Percent'].mean().sort_values(ascending=False)
    bars = plt.bar(category_avg.index, category_avg.values, color='teal', alpha=0.7)
    plt.xlabel('Food Category')
    plt.ylabel('Average YoY Inflation (%)')
    plt.title('Average Inflation by Food Category', fontweight='bold', fontsize=14)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_comprehensive_report(inflation_data):
    """Generate a comprehensive analysis report"""
    
    if inflation_data.empty:
        return "No data available for report"
    
    print("COMPREHENSIVE FOOD PRICE ANALYSIS REPORT")
    print("=" * 70)
    
    # Overall statistics
    avg_inflation = inflation_data['YoY_Percent'].mean()
    max_inflation = inflation_data['YoY_Percent'].max()
    min_inflation = inflation_data['YoY_Percent'].min()
    
    # Most and least inflated items
    most_inflated = inflation_data.nlargest(1, 'YoY_Percent').iloc[0]
    least_inflated = inflation_data.nsmallest(1, 'YoY_Percent').iloc[0]
    
    # Category with highest inflation
    category_inflation = inflation_data.groupby('Category')['YoY_Percent'].mean()
    highest_category = category_inflation.idxmax()
    highest_category_rate = category_inflation.max()
    
    print(f"\nOVERALL INFLATION:")
    print(f"   • Average YoY Inflation: {avg_inflation:.1f}%")
    print(f"   • Maximum Inflation: {max_inflation:.1f}%")
    print(f"   • Minimum Inflation: {min_inflation:.1f}%")
    
    print(f"\nEXTREMES:")
    print(f"   • Most Inflated: {most_inflated['Item']} ({most_inflated['YoY_Percent']:.1f}%)")
    print(f"     Price: ₦{most_inflated['Nov_2021']:,.0f} → ₦{most_inflated['Nov_2022']:,.0f}")
    print(f"   • Least Inflated: {least_inflated['Item']} ({least_inflated['YoY_Percent']:.1f}%)")
    
    print(f"\nCATEGORY ANALYSIS:")
    print(f"   • Highest Inflation Category: {highest_category} ({highest_category_rate:.1f}%)")
    
    # Items with over 20% inflation
    high_inflation_items = inflation_data[inflation_data['YoY_Percent'] > 20]
    print(f"\nCRITICAL ITEMS (>20% Inflation): {len(high_inflation_items)} items")
    for _, item in high_inflation_items.nlargest(5, 'YoY_Percent').iterrows():
        print(f"   • {item['Item']}: {item['YoY_Percent']:.1f}%")
    
    print(f"\nDATA COVERAGE:")
    print(f"   • Total Items Analyzed: {len(inflation_data)}")
    print(f"   • Categories Covered: {len(inflation_data['Category'].unique())}")

# Generate the final report
generate_comprehensive_report(inflation_results)

In [ ]:
# Let's analyze the top inflated items in more detail
print("DETAILED ANALYSIS OF TOP INFLATED ITEMS")
print("=" * 60)

if not inflation_results.empty:
    # Get top 10 most inflated items
    top_10 = inflation_results.nlargest(10, 'YoY_Percent')
    
    # Display with better formatting
    pd.set_option('display.float_format', '{:,.2f}'.format)
    
    print("Top 10 Most Inflated Items (Detailed):")
    detailed_top = top_10[['Item', 'Nov_2021', 'Nov_2022', 'YoY_Increase', 'YoY_Percent']].copy()
    detailed_top['Nov_2021'] = detailed_top['Nov_2021'].round(2)
    detailed_top['Nov_2022'] = detailed_top['Nov_2022'].round(2)
    detailed_top['YoY_Increase'] = detailed_top['YoY_Increase'].round(2)
    detailed_top['YoY_Percent'] = detailed_top['YoY_Percent'].round(2)
    
    display(detailed_top)
    
    # Summary statistics
    print(f"\nSUMMARY STATISTICS:")
    print(f"Average YoY Inflation: {inflation_results['YoY_Percent'].mean():.1f}%")
    print(f"Maximum YoY Inflation: {inflation_results['YoY_Percent'].max():.1f}%")
    print(f"Minimum YoY Inflation: {inflation_results['YoY_Percent'].min():.1f}%")
    print(f"Number of items analyzed: {len(inflation_results)}")
    
    # Count items by inflation category
    high_inflation = len(inflation_results[inflation_results['YoY_Percent'] > 20])
    medium_inflation = len(inflation_results[(inflation_results['YoY_Percent'] > 10) & (inflation_results['YoY_Percent'] <= 20)])
    low_inflation = len(inflation_results[inflation_results['YoY_Percent'] <= 10])
    
    print(f"\nINFLATION DISTRIBUTION:")
    print(f"High inflation (>20%): {high_inflation} items")
    print(f"Medium inflation (10-20%): {medium_inflation} items") 
    print(f"Low inflation (≤10%): {low_inflation} items")

In [ ]:
# Create more comprehensive visualizations
if not inflation_results.empty:
    # Create a figure with multiple subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
    
    # Plot 1: Top 10 Most Inflated Items
    top_10 = inflation_results.nlargest(10, 'YoY_Percent')
    bars1 = ax1.barh(top_10['Item'], top_10['YoY_Percent'], color='red', alpha=0.7)
    ax1.set_xlabel('YoY Inflation (%)')
    ax1.set_title('Top 10 Most Inflated Items', fontweight='bold', fontsize=14)
    ax1.invert_yaxis()
    for bar in bars1:
        width = bar.get_width()
        ax1.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontweight='bold')
    
    # Plot 2: Bottom 10 Least Inflated Items
    bottom_10 = inflation_results.nsmallest(10, 'YoY_Percent')
    bars2 = ax2.barh(bottom_10['Item'], bottom_10['YoY_Percent'], color='green', alpha=0.7)
    ax2.set_xlabel('YoY Inflation (%)')
    ax2.set_title('Top 10 Least Inflated Items', fontweight='bold', fontsize=14)
    ax2.invert_yaxis()
    for bar in bars2:
        width = bar.get_width()
        ax2.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.1f}%', ha='left', va='center', fontweight='bold')
    
    # Plot 3: Price Comparison for Top 5 Items
    top_5 = inflation_results.nlargest(5, 'YoY_Percent')
    x_pos = np.arange(len(top_5))
    width = 0.35
    
    ax3.bar(x_pos - width/2, top_5['Nov_2021'], width, label='Nov 2021', alpha=0.7)
    ax3.bar(x_pos + width/2, top_5['Nov_2022'], width, label='Nov 2022', alpha=0.7)
    ax3.set_xlabel('Items')
    ax3.set_ylabel('Price (₦)')
    ax3.set_title('Price Comparison: Nov 2021 vs Nov 2022', fontweight='bold', fontsize=14)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels([item[:15] + '...' if len(item) > 15 else item for item in top_5['Item']], rotation=45)
    ax3.legend()
    
    # Plot 4: Inflation Distribution Histogram
    ax4.hist(inflation_results['YoY_Percent'], bins=15, color='purple', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('YoY Inflation (%)')
    ax4.set_ylabel('Number of Items')
    ax4.set_title('Distribution of Inflation Rates', fontweight='bold', fontsize=14)
    ax4.axvline(inflation_results['YoY_Percent'].mean(), color='red', linestyle='--', 
                label=f'Mean: {inflation_results["YoY_Percent"].mean():.1f}%')
    ax4.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Let's categorize items and analyze by food type
def categorize_items(item_name):
    """Categorize food items into broad categories"""
    item_lower = str(item_name).lower()
    
    if any(word in item_lower for word in ['beef', 'chicken', 'fish', 'mackerel', 'sardine', 'tilapia']):
        return 'Animal Protein'
    elif any(word in item_lower for word in ['rice', 'beans', 'maize', 'wheat', 'flour']):
        return 'Grains & Staples'
    elif any(word in item_lower for word in ['oil', 'palm', 'groundnut', 'vegetable']):
        return 'Oils & Fats'
    elif any(word in item_lower for word in ['tomato', 'onion', 'plantain', 'yam', 'potato']):
        return 'Vegetables & Fruits'
    elif any(word in item_lower for word in ['egg', 'milk']):
        return 'Dairy & Eggs'
    elif any(word in item_lower for word in ['bread', 'gari']):
        return 'Processed Foods'
    else:
        return 'Other'

# Add categories to our inflation data
if not inflation_results.empty:
    inflation_results['Category'] = inflation_results['Item'].apply(categorize_items)
    
    # Analyze inflation by category
    category_analysis = inflation_results.groupby('Category').agg({
        'YoY_Percent': ['mean', 'max', 'min', 'count'],
        'Nov_2021': 'mean',
        'Nov_2022': 'mean'
    }).round(2)
    
    category_analysis.columns = ['Avg_YoY_%', 'Max_YoY_%', 'Min_YoY_%', 'Item_Count', 'Avg_Price_2021', 'Avg_Price_2022']
    category_analysis = category_analysis.sort_values('Avg_YoY_%', ascending=False)
    
    print("INFLATION BY FOOD CATEGORY:")
    display(category_analysis)
    
    # Visualization by category
    plt.figure(figsize=(12, 6))
    category_avg = inflation_results.groupby('Category')['YoY_Percent'].mean().sort_values(ascending=False)
    bars = plt.bar(category_avg.index, category_avg.values, color='teal', alpha=0.7)
    plt.xlabel('Food Category')
    plt.ylabel('Average YoY Inflation (%)')
    plt.title('Average Inflation by Food Category', fontweight='bold', fontsize=14)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_comprehensive_report(inflation_data):
    """Generate a comprehensive analysis report"""
    
    if inflation_data.empty:
        return "No data available for report"
    
    print("COMPREHENSIVE FOOD PRICE ANALYSIS REPORT")
    print("=" * 70)
    
    # Overall statistics
    avg_inflation = inflation_data['YoY_Percent'].mean()
    max_inflation = inflation_data['YoY_Percent'].max()
    min_inflation = inflation_data['YoY_Percent'].min()
    
    # Most and least inflated items
    most_inflated = inflation_data.nlargest(1, 'YoY_Percent').iloc[0]
    least_inflated = inflation_data.nsmallest(1, 'YoY_Percent').iloc[0]
    
    # Category with highest inflation
    category_inflation = inflation_data.groupby('Category')['YoY_Percent'].mean()
    highest_category = category_inflation.idxmax()
    highest_category_rate = category_inflation.max()
    
    print(f"\nOVERALL INFLATION:")
    print(f"   • Average YoY Inflation: {avg_inflation:.1f}%")
    print(f"   • Maximum Inflation: {max_inflation:.1f}%")
    print(f"   • Minimum Inflation: {min_inflation:.1f}%")
    
    print(f"\nEXTREMES:")
    print(f"   • Most Inflated: {most_inflated['Item']} ({most_inflated['YoY_Percent']:.1f}%)")
    print(f"     Price: ₦{most_inflated['Nov_2021']:,.0f} → ₦{most_inflated['Nov_2022']:,.0f}")
    print(f"   • Least Inflated: {least_inflated['Item']} ({least_inflated['YoY_Percent']:.1f}%)")
    
    print(f"\nCATEGORY ANALYSIS:")
    print(f"   • Highest Inflation Category: {highest_category} ({highest_category_rate:.1f}%)")
    
    # Items with over 20% inflation
    high_inflation_items = inflation_data[inflation_data['YoY_Percent'] > 20]
    print(f"\nCRITICAL ITEMS (>20% Inflation): {len(high_inflation_items)} items")
    for _, item in high_inflation_items.nlargest(5, 'YoY_Percent').iterrows():
        print(f"   • {item['Item']}: {item['YoY_Percent']:.1f}%")
    
    print(f"\nDATA COVERAGE:")
    print(f"   • Total Items Analyzed: {len(inflation_data)}")
    print(f"   • Categories Covered: {len(inflation_data['Category'].unique())}")

# Generate the final report
generate_comprehensive_report(inflation_results)

 CRITICAL INSIGHTS FROM THE ANALYSIS:

Red Alert Findings:

Processed Foods Crisis: Bread prices surged 30-34% - this hits low-income families hardest as 
bread is a staple breakfast item.

Protein Inflation: Beef prices up 28-29% - making basic nutrition unaffordable for many households.

Egg Price Shock: 41.2% increase in egg prices - this is particularly alarming as eggs 
are a primary protein source for millions.


In [ ]:
# Focus on the crisis items
crisis_items = ['Agric eggs(medium size price of one)', 'Bread unsliced 500g', 
                'Bread sliced 500g', 'Beef,boneless', 'Beef Bone in']

crisis_analysis = inflation_results[inflation_results['Item'].isin(crisis_items)].copy()
crisis_analysis = crisis_analysis.sort_values('YoY_Percent', ascending=False)

print("CRISIS ITEMS - DETAILED IMPACT ANALYSIS")
print("=" * 65)

for _, item in crisis_analysis.iterrows():
    absolute_increase = item['YoY_Increase']
    percent_increase = item['YoY_Percent']
    
    print(f"\n{item['Item']}:")
    print(f"Nov 2021: ₦{item['Nov_2021']:,.0f}")
    print(f"Nov 2022: ₦{item['Nov_2022']:,.0f}")
    print(f"Absolute Increase: ₦{absolute_increase:,.0f}")
    print(f"Percentage Increase: {percent_increase:.1f}%")
    
    # Calculate monthly impact on household budget
    if "egg" in item['Item'].lower():
        monthly_consumption = 30  # Assuming 1 egg per day
        monthly_cost_2021 = item['Nov_2021'] * monthly_consumption
        monthly_cost_2022 = item['Nov_2022'] * monthly_consumption
        monthly_increase = monthly_cost_2022 - monthly_cost_2021
        print(f"Monthly Impact (30 eggs): ₦{monthly_increase:,.0f} additional")
    
    elif "bread" in item['Item'].lower():
        monthly_consumption = 8  # Assuming 2 loaves per week
        monthly_cost_2021 = item['Nov_2021'] * monthly_consumption
        monthly_cost_2022 = item['Nov_2022'] * monthly_consumption
        monthly_increase = monthly_cost_2022 - monthly_cost_2021
        print(f"Monthly Impact (8 loaves): ₦{monthly_increase:,.0f} additional")
    
    elif "beef" in item['Item'].lower():
        monthly_consumption = 4  # Assuming 1kg per week
        monthly_cost_2021 = item['Nov_2021'] * monthly_consumption
        monthly_cost_2022 = item['Nov_2022'] * monthly_consumption
        monthly_increase = monthly_cost_2022 - monthly_cost_2021
        print(f"Monthly Impact (4kg beef): ₦{monthly_increase:,.0f} additional")

In [ ]:
# Calculate the total impact on a typical household budget
def calculate_household_impact(inflation_data):
    """Calculate the monthly impact on household budgets"""
    
    # Typical monthly consumption patterns for a family of 4
    consumption_pattern = {
        'Agric eggs(medium size price of one)': 30,  # 1 egg per day
        'Bread sliced 500g': 8,  # 2 loaves per week
        'Bread unsliced 500g': 4,  # 1 loaf per week
        'Beef,boneless': 2,  # 0.5kg per week
        'Beef Bone in': 2,   # 0.5kg per week
    }
    
    print("HOUSEHOLD BUDGET IMPACT ANALYSIS")
    print("=" * 60)
    print("Assumption: Family of 4, typical consumption patterns\n")
    
    total_monthly_increase = 0
    detailed_impact = []
    
    for item, monthly_qty in consumption_pattern.items():
        item_data = inflation_data[inflation_data['Item'] == item]
        if not item_data.empty:
            item_row = item_data.iloc[0]
            monthly_cost_2021 = item_row['Nov_2021'] * monthly_qty
            monthly_cost_2022 = item_row['Nov_2022'] * monthly_qty
            monthly_increase = monthly_cost_2022 - monthly_cost_2021
            
            detailed_impact.append({
                'Item': item,
                'Monthly Qty': monthly_qty,
                'Monthly Cost 2021': monthly_cost_2021,
                'Monthly Cost 2022': monthly_cost_2022,
                'Monthly Increase': monthly_increase
            })
            
            total_monthly_increase += monthly_increase
    
    # Create impact dataframe
    impact_df = pd.DataFrame(detailed_impact)
    
    # Display results
    for impact in detailed_impact:
        print(f"{impact['Item'][:25]}...:")
        print(f"Monthly Quantity: {impact['Monthly Qty']}")
        print(f"2021 Monthly Cost: ₦{impact['Monthly Cost 2021']:,.0f}")
        print(f"2022 Monthly Cost: ₦{impact['Monthly Cost 2022']:,.0f}")
        print(f"Additional Monthly: ₦{impact['Monthly Increase']:,.0f}")
        print()
    
    print(f"TOTAL ADDITIONAL MONTHLY COST: ₦{total_monthly_increase:,.0f}")
    print(f"TOTAL ADDITIONAL ANNUAL COST: ₦{total_monthly_increase * 12:,.0f}")
    
    return impact_df, total_monthly_increase

# Calculate household impact
household_impact, total_increase = calculate_household_impact(inflation_results)

In [ ]:
def generate_policy_recommendations(inflation_data, household_impact):
    """Generate targeted policy recommendations based on the analysis"""
    
    print("POLICY RECOMMENDATIONS & PRIORITY ACTIONS")
    print("=" * 65)
    
    # Identify the most critical items
    crisis_items = inflation_data.nlargest(5, 'YoY_Percent')
    
    print("\nIMMEDIATE PRIORITY ACTIONS (Next 30 days):")
    
    # Egg crisis
    egg_items = inflation_data[inflation_data['Item'].str.contains('egg', case=False, na=False)]
    if not egg_items.empty:
        egg_inflation = egg_items['YoY_Percent'].iloc[0]
        print(f"\nEGG PRICE CRISIS ({egg_inflation:.1f}% increase):")
        print("Investigate poultry feed supply chain")
        print("Provide temporary subsidies for small-scale poultry farmers")
        print("Import duty waivers on chicken feed ingredients")
    
    # Bread crisis
    bread_items = inflation_data[inflation_data['Item'].str.contains('bread', case=False, na=False)]
    if not bread_items.empty:
        bread_inflation = bread_items['YoY_Percent'].mean()
        print(f"\n2.BREAD PRICE CRISIS ({bread_inflation:.1f}% average increase):")
        print("Review wheat import policies and tariffs")
        print("Support local flour millers with energy cost subsidies")
        print("Monitor and regulate bakeries' profit margins")
    
    # Beef crisis
    beef_items = inflation_data[inflation_data['Item'].str.contains('beef', case=False, na=False)]
    if not beef_items.empty:
        beef_inflation = beef_items['YoY_Percent'].mean()
        print(f"\nBEEF PRICE CRISIS ({beef_inflation:.1f}% average increase):")
        print("Enhance livestock disease control programs")
        print("Improve cattle transportation infrastructure")
        print("Support alternative protein sources (fish, poultry)")
    
    print(f"\nSTRATEGIC RECOMMENDATIONS:")
    print(f"Target inflation reduction to below 15% for essential foods")
    print(f"Establish price monitoring and early warning system")
    print(f"Support local production of critical items")
    print(f"Consider temporary VAT removal on essential food items")
    
    print(f"\nECONOMIC IMPACT:")
    print(f"Average household faces ₦{total_increase:,.0f} additional monthly food costs")
    print(f"This represents a significant reduction in disposable income")
    print(f"Food security at risk for low-income households")

# Generate policy recommendations
generate_policy_recommendations(inflation_results, household_impact)

In [ ]:
def risk_assessment_and_forecast(inflation_data):
    """Assess risks and provide short-term forecast"""
    
    print("RISK ASSESSMENT & 6-MONTH OUTLOOK")
    print("=" * 60)
    
    current_avg_inflation = inflation_data['YoY_Percent'].mean()
    
    print(f"\nCURRENT SITUATION:")
    print(f"Average Food Inflation: {current_avg_inflation:.1f}%")
    print(f"Crisis Level: {'HIGH ALERT' if current_avg_inflation > 20 else 'MODERATE'}")
    print(f"Items in Crisis (>20% inflation): {len(inflation_data[inflation_data['YoY_Percent'] > 20])}")
    
    print(f"\nRISK FACTORS:")
    print(f"Global commodity prices continue to rise")
    print(f"Supply chain disruptions persist")
    print(f"Currency exchange rate pressures")
    print(f"Seasonal factors (dry season approaching)")
    
    print(f"\n6-MONTH PROJECTION:")
    if current_avg_inflation > 25:
        print(f"Outlook: CONTINUED HIGH INFLATION LIKELY")
        print(f"Expected range: 20-30% average inflation")
        print(f"Risk of broader economic impact: HIGH")
    else:
        print(f"Outlook: MODERATE IMPROVEMENT POSSIBLE")
        print(f"Expected range: 15-25% average inflation")
    
    print(f"\nCRITICAL MONITORING ITEMS:")
    critical_items = inflation_data.nlargest(3, 'YoY_Percent')['Item'].tolist()
    for i, item in enumerate(critical_items, 1):
        print(f"   {i}. {item}")

# Conduct risk assessment
risk_assessment_and_forecast(inflation_results)

Executive Summary:
Based on the analysis, here's the bottom line:
CRISIS LEVEL: HIGH
25.9% average food inflation is economically unsustainable
6 out of 10 items show crisis-level inflation (>20%)
Processed foods are the worst hit (32.6% average)

Household Impact: Severe
Typical family faces thousands of Naira in additional monthly food costs
Protein sources becoming unaffordable for average households
Food security at risk for vulnerable populations

Immediate Priorities:
Address egg supply chain (41.2% inflation)
Stabilize bread prices (30-34% inflation) 
Control beef costs (28-29% inflation)

This analysis provides a clear roadmap for policymakers and business leaders to address the 
most critical food inflation challenges!

In [ ]:
import os
import json
from datetime import datetime

# Create project directory structure
project_name = "Nigeria-Food-Price-Analysis-2022"
folders = ['data', 'analysis', 'visualizations', 'reports', 'docs']

print("CREATING PROJECT STRUCTURE...")

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    print(f"Created folder: {folder}/")

print("\nProject structure created successfully!")

In [ ]:
# Save cleaned and processed data
def save_processed_data(inflation_data, household_impact, main_data):
    """Save all processed data to CSV files"""
    
    print("SAVING PROCESSED DATA...")
    
    # Save inflation analysis
    inflation_data.to_csv('data/inflation_analysis.csv', index=False)
    print("Saved: data/inflation_analysis.csv")
    
    # Save household impact analysis
    household_impact.to_csv('data/household_impact.csv', index=False)
    print("Saved: data/household_impact.csv")

In [4]:
# Create basic notebook file
notebook_content = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# Nigeria Food Price Analysis 2022"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}

import json
with open('analysis/food_price_analysis.ipynb', 'w') as f:
    json.dump(notebook_content, f)

print("Basic notebook created: analysis/food_price_analysis.ipynb")


Basic notebook created: analysis/food_price_analysis.ipynb
